In [1]:
from narupa.app.client import NarupaClient
from narupa.ase.converter import frame_data_to_ase
from narupa.trajectory.frame_data import (
    FrameData, POSITIONS, ELEMENTS, TYPES, MissingDataError,
)
from narupa.imd.interaction import Interaction
import numpy as np
import nglview
import MDAnalysis as mda
import matplotlib.pyplot as plt
from queue import Queue

## Start an IMD client

In [16]:
class NGLClient(NarupaClient):
    def __init__(self, dynamic_bonds=False, *args, update_callback=None, **kwargs):
        super().__init__(*args, **kwargs)
        self._view = None
        self.update_callback = update_callback
        self.dynamic_bonds = dynamic_bonds
    
    @property
    def view(self):
        if self._view is None or self.dynamic_bonds:
            atoms = frame_data_to_ase(
                client.first_frame,
                topology=True,
                positions=False,
            )
            atoms.set_positions(
                np.array(client.latest_frame.particle_positions) * 10
            )
            self._view = nglview.show_ase(atoms)
        return self._view

    def _on_frame_received(self, frame_index:int, frame):
        super()._on_frame_received(frame_index, frame)
        self.view.set_coordinates(
            {0: np.array(self.latest_frame.particle_positions) * 10}
        )
        if self.update_callback is not None:
            self.update_callback(self.universe)

In [17]:
client = NGLClient()

In [20]:
positions = np.array(client.latest_frame.particle_positions) * 10
start_point = positions[0, :]
first_anchor = positions[0] - np.array([0.1,0,0])

In [19]:
anchor1 = Interaction(position=first_anchor, scale=1000, particles=(0,), interaction_type='spring')
anchor1_id = client.start_interaction()
client.update_interaction(anchor1_id, anchor1)

In [22]:
client.view

NGLWidget(n_components=1)

In [8]:
client.view.shape.add_sphere(list(start_point), [1, 0, 0], 0.5)